In [1]:
import pandas as pd 
import numpy as np
import dill


In [32]:
X_train= pd.read_csv('dataset/X_train_Final.csv')
y_train= pd.read_csv('dataset/y_train.csv')

In [33]:
X_test= pd.read_csv('dataset/X_test_Final.csv')
y_test= pd.read_csv('dataset/y_test.csv')

In [9]:
y_test_id = X_test["id"]

In [34]:
X_train=X_train.drop("Sale_Price", axis=1)
X_test=X_test.drop("Sale_Price", axis=1)

In [35]:
X_train=X_train.drop("id", axis=1)
X_test=X_test.drop("id", axis=1)

In [36]:
y_train=y_train.squeeze() #converte in series
y_test=y_test.squeeze() #converte in series

In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from datetime import datetime

In [8]:
results = pd.DataFrame()

In [4]:
results = pd.read_csv('results/prediction.csv')

In [38]:
def model_evaluation(model_name, model ,X_train, y_train, X_test, y_test, log_scale):
    
    if log_scale is True :
        model.fit(X_train, np.log(y_train))
        y_predicted = np.exp(model.predict(X_test))
        
    else:
        model.fit(X_train, np.log(y_train))
        y_predicted = np.exp(model.predict(X_test))
    
    mse = mean_squared_error(y_test, y_predicted)
    rmse = mse**.5
    r2 = r2_score(y_test, y_predicted)
    mae = mean_absolute_error(y_test, y_predicted)
    
    print(f'Model Name : {model_name}')
    print(f'RMSE : {rmse}')
    print(f'R2 : {r2}')
    print(f'MAE : {mae}')
    
    results[model_name] = y_predicted   

# RANDOM FOREST REGRESSOR

In [31]:
rf = RandomForestRegressor()
model_evaluation("Simple RandomForest", rf , X_train, y_train, X_test, y_test, False )

Model Name : Simple RandomForest
RMSE : 23358.91078368234
R2 : 0.9231035598861133
MAE : 15654.360363221123


## FACCIAMO TUNING DEI PARAMETRI DELLA RANDOM FOREST

In [28]:
## Define Grid 
param_grid = {'n_estimators': [2000,4000,5000], #numero di alberi nella foresta
              'min_samples_split': [2,4,6], #numero minimo di campioni in un nodo interno per permettere lo split
              'bootstrap' : [True], #campioni con reinserimento
              'max_depth' : [7,15,None], #estensione del'albero, None --> estensione fino a foglie pure
              'max_features': [0.3], #numero di feature su cui sono allenati gli alberi
              } 

grid_search = model_selection.GridSearchCV(rf, #modello utilizzato
                                           param_grid, #griglia dei parametri
                                           scoring="neg_root_mean_squared_error", #MSE MEAN SQUARE ERROR
                                           verbose=3,
                                           n_jobs=-1, #multi processor
                                           #cv = 5 <-- default cross validation
                                           return_train_score=True)
## show start time
print(datetime.now())
best_forest = grid_search.fit(X_train, y_train)#Allena tanti modelli quante sono le combinazioni tra i parametri definiti in param_grid
print('Optimum parameters', best_forest.best_params_)
## show end time
print(datetime.now())

2022-12-30 13:55:34.254160
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Optimum parameters {'bootstrap': True, 'max_depth': None, 'max_features': 0.3, 'min_samples_split': 4, 'n_estimators': 5000}
2022-12-30 14:14:46.683673


## 1° TUNING
- 'n_estimators': [500,1000,2000,4000], #numero di alberi nella foresta
- 'min_samples_split': [2], #numero minimo di campioni in un nodo interno per permettere lo split
- 'bootstrap' : [True], #campioni con reinserimento
- 'max_depth' : [5,6,7,None], #estensione del'albero, None --> estensione fino a foglie pure
- 'max_features': ['sqrt','log2', 0.3], #numero di feature su cui sono allenati gli alberi

2022-12-29 22:39:12.899880  
Fitting 5 folds for each of 48 candidates, totalling 240 fits

**Optimum parameters {'bootstrap': True, 'max_depth': None, 'max_features': 0.3, 'min_samples_split': 2, 'n_estimators': 2000}**  
2022-12-29 22:50:10.791188


In [33]:
rf_tuned_1 = RandomForestRegressor(bootstrap=True, max_depth=None, min_samples_split=2, n_estimators=2000, max_features=0.3)
model_evaluation("First Tuned RandomForest", rf_tuned_1, X_train, y_train, X_test, y_test, False)

Model Name : First Tuned RandomForest
RMSE : 23408.45230560886
R2 : 0.9227770372556441
MAE : 15300.688261848281


In [14]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'vscode+colab' #permette il rendering dei plot sia su vscode che su colab
import matplotlib.pyplot as plt


## Tuned Random Forest - Visualizziamo i valori predetti vs i valori reali osservati

In [25]:
fig =px.scatter(results, x="Observed", y="First Tuned RandomForest", hover_data=['id'],color_discrete_sequence=['red'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.show()

# 2° TUNING
- 'n_estimators': [2000,4000,5000], #numero di alberi nella foresta
- 'min_samples_split': [2,4,6], #numero minimo di campioni in un nodo interno per permettere lo split
- 'bootstrap' : [True], #campioni con reinserimento
- 'max_depth' : [7,15,None], #estensione del'albero, None --> estensione fino a foglie pure
- 'max_features': [0.3], #numero di feature su cui sono allenati gli alberi

2022-12-30 13:55:34.254160  
Fitting 5 folds for each of 27 candidates, totalling 135 fits  
  
**Optimum parameters {'bootstrap': True, 'max_depth': None, 'max_features': 0.3, 'min_samples_split': 4, 'n_estimators': 5000}**
2022-12-30 14:14:46.683673

In [39]:
rf_tuned_2 = RandomForestRegressor(bootstrap=True, max_depth=None, min_samples_split=4, n_estimators=5000, max_features=0.3)
model_evaluation("Second Tuned RandomForest", rf_tuned_2, X_train, y_train, X_test, y_test, False)

Model Name : Second Tuned RandomForest
RMSE : 23413.459868120375
R2 : 0.9227439944752209
MAE : 15298.484457495555


## 2nd Tuned Random Forest - Visualizziamo i valori predetti vs i valori reali osservati

In [28]:
fig =px.scatter(results, x="Observed", y="Second Tuned RandomForest", hover_data=['id'], color_discrete_sequence=['red'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.show()

# TRASFORMAZIONE LOG E PROVA DELLA RANDOM FOREST SU DATASET CON OUTLIERS

A questo punto come avevamo constatato in fase di visualizzazione della distribuzione di SalePrice proviamo a transformala in scala logaritmica e vediamo se la nostra miglior random forest avrà un RMSE migliore.

In [41]:
y_train_log = np.log(y_train) #trasformazione logaritmica della variabile risposta di train
y_test_log = np.log(y_test) #trasformazione logaritmica della variabile risposta di test

In [43]:
rf_log = RandomForestRegressor(bootstrap=True, max_depth=None, min_samples_split=2, n_estimators=5000, max_features=0.3)
model_evaluation("Log RandomForest", rf_log, X_train, y_train, X_test, y_test, True)

Model Name : Log RandomForest
RMSE : 23356.118093679564
R2 : 0.9231219455963358
MAE : 15281.695314150791


## Dataset con outliers

In [44]:
X_train_out = pd.read_csv('dataset/train_data_FE_encoded_With_outliers.csv') #lettura dataset con outliers rimossi con IsolationForest
y_train_out = pd.read_csv('dataset/y_train_With_outliers.csv').squeeze()
X_train_out=X_train_out.drop(["Sale_Price", "id"], axis=1)

In [45]:
rf_out = RandomForestRegressor(bootstrap=True, max_depth=None, min_samples_split=2, n_estimators=5000, max_features=0.3)
model_evaluation("Outliers RandomForest", rf_out, X_train_out, y_train_out, X_test, y_test, False)


Model Name : Outliers RandomForest
RMSE : 23170.09052717487
R2 : 0.9243417102032093
MAE : 15133.887252079814


## Outliers Random Forest - Visualizziamo i valori predetti vs i valori reali osservati

In [29]:
fig =px.scatter(results, x="Observed", y="Outliers RandomForest", hover_data=["id"],color_discrete_sequence=['red'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.show()

In [12]:
results.to_csv('results/prediction.csv', index=False)

In [11]:
#results = results[['id','Observed','Simple RandomForest','First Tuned RandomForest','Second Tuned RandomForest', 'Log RandomForest', 'Outliers RandomForest']]

Il modelli non sembrano essere migliorati molto rispetto alle altre RandomForest.  
Questo ci fa capire che essendo le random forest basate su alberi decisionali indipendenti, addestrati su campioni casuali sia delle osservazioni che delle variabili, non necessitano di una trasformazione della variabile risposta. Grazie al Bagging quindi la random forest risulta essere robusta ai valori outliers e stabile a prescindere dalle trasformazioni delle variabili

# XGBOOST REGRESSOR

In [31]:
import xgboost as xgbx
from xgboost.sklearn import XGBRegressor

# SIMPLE XGBOOST REGRESSOR

In [43]:
xgb_model = XGBRegressor(learning_rate = 0.1)
model_evaluation("Simple XGBRegressor", xgb_model ,X_train, y_train, X_test, y_test, False)

c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



Model Name : Simple XGBRegressor
RMSE : 22388.985014475802
R2 : 0.9293568802711545
MAE : 14228.529496717258


In [99]:
fig =px.scatter(results, x="Observed", y="Simple XGBRegressor", hover_data=["id"],color_discrete_sequence=['green'],trendline="ols",trendline_color_override='red',height=1000, width=1000)
fig.show()

# TUNING DEI PARAMETRI DI XGBOOST

In [45]:
xgb_model_tuned = XGBRegressor()

In [107]:

param_grid = { 
              'n_estimators':[5000],#numero di alberi
              'max_depth' : [3,10,None], #estensione del'albero, None --> estensione fino a foglie pure
              'colsample_bytree': [0.3, 0.7], #numero di feature su cui sono allenati gli alberi
              'learning_rate' : [0.01,0.05, 0.1]
              } 

grid_search = model_selection.GridSearchCV(xgb_model_tuned, #modello utilizzato
                                           param_grid, #griglia dei parametri
                                           scoring=["r2","neg_root_mean_squared_error"], #MSE MEAN SQUARE ERROR
                                           refit="neg_root_mean_squared_error", #In the multi-metric setting, you need to set this so that the final model can be fitted to that,
                                                                                #because the best hyper-parameters for the model will be decided based on a single metric only.
                                           verbose=1,
                                           n_jobs=-1, #multi processor
                                           #cv = 5 <-- default cross validation
                                           return_train_score=True)
## show start time
print(datetime.now())
best_model = grid_search.fit(X_train, y_train)#Allena tanti modelli quante sono le combinazioni tra i parametri definiti in param_grid
print('Optimum parameters', best_model.best_params_)
print('Scores:', (best_model.best_score_))
## show end time
print(datetime.now())

2022-12-31 16:26:18.121544
Fitting 5 folds for each of 18 candidates, totalling 90 fits


c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



Optimum parameters {'colsample_bytree': 0.3, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 5000}
Higher R2 -20197.110230697854
2022-12-31 16:48:57.856716


In [114]:
xgb_model_tuned = best_model.best_estimator_
model_evaluation("Tuned XGBRegressor", xgb_model_tuned, X_train, y_train, X_test, y_test, False)

c:\Users\Marco\Projects\house_prediction\conda-env\lib\site-packages\xgboost\data.py:250: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



Model Name : Tuned XGBRegressor
RMSE : 20375.812575003853
R2 : 0.9414898874889327
MAE : 13504.83685943895


In [111]:
fig =px.scatter(results, x="Observed", y="Tuned XGBRegressor", hover_data=["id"],color_discrete_sequence=['green'],trendline="ols",trendline_color_override='blue',height=1000, width=1000)
fig.show()

Notiamo che xgb_model_tuned riesce ad avere predizioni migliori sulle case con prezzo più alto

In [11]:
import dill

def save_session() :
    dill.dump_session('data_prediction.db')
    
def load_session():
    dill.load_session('data_prediction.db')

In [84]:
#save_session()

In [12]:
#load_session()